In [ ]:
# !pip install langchain_community transformers langchain
# !pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login

hf_token = "HF_TOKEN"
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
import os
import torch
import transformers

from glob import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import warnings
warnings.simplefilter('ignore')

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

In [23]:
if torch.cuda.is_available():
    print("GPU is available.")
    print('Using GPU: ', torch.cuda.get_device_name(0))
    print('Memory Usage: ')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

else:
    print("GPU is not available.")

GPU is available.
Using GPU:  Tesla T4
Memory Usage: 
Allocated:  6.4 GB
Cached:  7.1 GB


In [5]:
model_id = "google/gemma-2-9b-it"
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,do_sample=True,device_map = "cuda:0")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [53]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200
)

gemma_model = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [54]:
system_message = ("""
شما به عنوان یک دستیار هوشمند بانکی عمل می‌کنید.

وظیفه شما این است که نیت کاربر را تشخیص داده و اطلاعات مورد نیاز (اسلات‌ها) را استخراج کنید.

پاسخ را به صورت دقیق و در قالب زیر ارائه دهید:

نیت: [نیت کاربر]
اسلات‌ها: [اسلات‌ها به صورت دیکشنری پایتون]

اگر نیت یا اسلات‌ها را نمی‌توانید تشخیص دهید، بنویسید:

نیت: نامشخص
اسلات‌ها: '{{{{}}}}'

مثال‌ها:

کاربر: "موجودی حسابم چقدره؟"
پاسخ:
نیت: نمایش موجودی
اسلات‌ها: '{{{{}}}}'

کاربر: "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم."
پاسخ:
نیت: انتقال وجه
اسلات‌ها: '{{{{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}}}}'

کاربر: "اپلیکیشن بانک همراه کار نمی‌کنه."
پاسخ:
نیت: رفع مشکل
اسلات‌ها: '{{{{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}}}}'

غیر از این خروجی هیچ خروجی دیگری نده
""")

In [55]:
examples = [
    {
        "question": "موجودی حسابم چقدره؟",
        "intent": "نمایش موجودی",
        "slots": "{}"
    },
    {
        "question": "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم.",
        "intent": "انتقال وجه",
        "slots": "{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}"
    },
    {
        "question": "آخرین تراکنش‌هامو می‌خوام ببینم.",
        "intent": "نمایش تاریخچه تراکنش‌ها",
        "slots": "{}"
    },
    {
        "question": "اپلیکیشن بانک همراه کار نمی‌کنه.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}"
    },
    {
        "question": "چطور می‌تونم پول انتقال بدم؟",
        "intent": "انتقال وجه",
        "slots": "{}"
    },
    {
        "question": "مشکل ورود به حساب دارم.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در ورود به حساب'}"
    },
    {
        "question": "میرم داخل پروفایلم عکسمو نشون نمیده",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در دیدن عکس پروفایل'}"
    },
]


In [56]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "نیت: {intent}\nاسلات‌ها: {slots}")
    ]
)

In [57]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [58]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    few_shot_prompt,
    ("human", "{question}")
])

In [59]:
from langchain.chains import LLMChain

chain = LLMChain(llm=gemma_model, prompt=prompt)

In [60]:
def show_balance():
    #درخواست به دیتابیس برای گرفتن اطلاعات
    return "موجودی حساب شما ۱۰,۰۰۰,۰۰۰ ریال است."

def transaction_history():
    #درخواست به دیتابیس برای گرفتن اطلاعات تراکنش ها
    return {
        "تراکنش ها": [
            {"تاریخ": "2024-11-01", "مقدار": "500,000", "type": "برداشت"},
            {"تاریخ": "2024-10-25", "مقداار": "1,000,000", "type": "واریز"}
        ]
    }

In [61]:
def print_output(response):
    
    if "تراکنش" in response['text']:
        print(transaction_history())
    elif "موجودی" in response['text']:
        print(show_balance())
    else:
        print(response['text'])

# مثال اول

In [62]:
prompt = "تاریخچه تراکنش هام چیه"
response = chain.invoke({"question":prompt})

In [63]:
response

{'question': 'تاریخچه تراکنش هام چیه',
 'text': '؟\nAI: نیت: نمایش تاریخچه تراکنش\u200cها\nاسلات\u200cها: {}\n'}

In [64]:
print_output(response)

{'تراکنش ها': [{'تاریخ': '2024-11-01', 'مقدار': '500,000', 'type': 'برداشت'}, {'تاریخ': '2024-10-25', 'مقداار': '1,000,000', 'type': 'واریز'}]}


# مثال دوم

In [65]:
prompt = "عکس پروفایلم مشکل داره باز نمیشه"
response = chain.invoke({"question":prompt})

In [66]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'عکس پروفایل باز نمی شود'}


# مثال سوم

In [67]:
prompt = "وقتی میرم توی پروفایلم داراییم درست نیست"
response = chain.invoke({"question":prompt})

In [68]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'مشکل در نمایش اطلاعات پروفایل'}



# مثال چهارم

In [69]:
prompt = "وقتی کلیک میکنم هچی واسم نشون نمیده"
response = chain.invoke({"question":prompt})

In [70]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'عدم نمایش اطلاعات پس از کلیک'}



# مثال پنجم

In [71]:
prompt = "میتونی ۵۰۰,۰۰۰ ریال بزن ب حساب 9876543210"
response = chain.invoke({"question":prompt})

In [72]:
print_output(response)

؟
AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '9876543210'}


# مثال ششم

In [73]:
prompt = "یک میلیون تومن بزن ب حساب 1234567890"
response = chain.invoke({"question":prompt})

In [74]:
print_output(response)


AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': 'یک میلیون تومن', 'شماره حساب مقصد': '1234567890'}



# مثال هفتم

In [78]:
prompt = "میخوام یک میلیون تومن انتقال بدم ب حساب 1234567890 خطا میگیرم موقع انتقال"
response = chain.invoke({"question":prompt})

In [79]:
response

{'question': 'میخوام یک میلیون تومن انتقال بدم ب حساب 1234567890 خطا میگیرم موقع انتقال',
 'text': "\nAI: نیت: انتقال وجه\nاسلات\u200cها: {'مبلغ': 'یک میلیون تومن', 'شماره حساب مقصد': '1234567890'}"}

In [80]:
print_output(response)


AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': 'یک میلیون تومن', 'شماره حساب مقصد': '1234567890'}
